# Basic tutorial: tabular data
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functioning of *giotto-deep* API. As an example, we take a simple classification problem in which we have a point cloud representing two entangled tori.

## Scope

The goal of this problem is to build a simple feed-forward neural network that is able to separate the two point clouds. The topology of the dataset is non trivial and we empirically discovered that a minimum number of nodes per layer is required to have a good splitting.

## Plan for this tutorial

The main steps of the tutorial are the following:
 1. creation of a dataset
 2. creation of a model
 3. define metrics and losses
 4. run trainig
 5. visualise results interactively

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import plotly.express as px
import torch
from torch import nn
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim import SGD, Adam
from torch.utils.data import SubsetRandomSampler
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_betti_surfaces
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from sklearn import datasets

from gdeep.models import FFNet
from gdeep.models import ModelExtractor
from gdeep.analysis.interpretability import Interpreter
from gdeep.utility.optimization import SAMOptimizer
from gdeep.visualization import persistence_diagrams_of_activations
from gdeep.trainer import Trainer
from gdeep.data.datasets import DatasetBuilder, DataLoaderBuilder
from gdeep.visualization import Visualiser
from gdeep.utility import DEVICE
from gdeep.search import GiottoSummaryWriter

# Initialize the tensorboard writer

In order to analyse the results of your models, you need to start tensorboard.
On the terminal, move inside the `/exampled` folder. There you can run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training pahse to see all the visualization results.

In [ ]:
writer = GiottoSummaryWriter()


# Create your dataset

The dataset consists of two entangled tori and the goal is to classify the points that belong to one or the other torus. Note that the topology of the decision boundary is non trivial!



In [ ]:
bd = DatasetBuilder(name="DoubleTori")
ds_tr, ds_val, _ = bd.build()

dl_builder = DataLoaderBuilder((ds_tr, ds_val))
dl_tr, dl_val, dl_ts = dl_builder.build(({"batch_size": 23}, {"batch_size": 23}))


## Define your model

In the next few cells we show hoe easy it is in giotto-deep to define a model and start th training: first, let's define the model using pytorch.

In [ ]:
class model1(nn.Module):
    def __init__(self):
        super(model1, self).__init__()
        self.seqmodel = nn.Sequential(nn.Flatten(), FFNet(arch=[3, 5, 10, 5, 2]))

    def forward(self, x):
        return self.seqmodel(x)


model = model1()


And now that we have the model and the data, only a couple of code lines is needed to start the training!

In [ ]:
# initlaise the loss function
loss_fn = nn.CrossEntropyLoss()

# initialise the pipelien class
pipe = Trainer(
    model,
    (dl_tr, dl_val),
    loss_fn,
    writer,
    k_fold_class=StratifiedKFold(3, shuffle=True),
)

# initialise the SAM optimiser
Optim = SAMOptimizer(SGD)  # this is a class, not an instance!

# train the model with learning rate scheduler and cross-validation
pipe.train(Optim, 5, True,
           optimizers_param={"lr": 0.01},
           lr_scheduler=ExponentialLR,
           scheduler_params={"gamma": 0.9},
           profiling=False,
           store_grad_layer_hist=True,
           writer_tag="tori")


## Evaluation

We can easily evaluate the model performance and compute the confusion matrix:

In [ ]:
# evaluation of the model performances for the classification task
pipe.evaluate_classification(2)


# Advanced topic: add pipeline hooks

It is possible to add a hook (a callable) that is called at the end of each training epoch.

The arguments of the hook are fix and have to respect the order you see in this example!

In [ ]:
def example_of_hook(epoch, optim, me, writer):
    print(
        f"Here we print the learning rate {optim.param_groups[0]['lr']} at epoch={epoch}"
    )
    print(
        f"We can also get the value of gradients and parameters of the model "
        f"using the model extractor! {me.get_layers_param():}"
    )


# register the hook
pipe.register_pipe_hook(example_of_hook)


Let's train the model with cross validation: we just have to set the parameter `cross_validation = True`.

The `keep_training = True` flag allow us to restart from the same scheduler, optimiser and trained model obtained at thhe end of the last training in the instance of the class `pipe`.

In [ ]:
# train the model with CV
pipe.train(
    SGD,
    3,
    cross_validation=True,
    keep_training=True,
    profiling=True,
    writer_tag="tori/kt",
)

# since we used the keep training flag, the optimiser has not been modified compared to the previous training.
print(pipe.optimizer)


# Simply use interpretability tools

Eventually, one would like to know why a neural network has made e certain classification choice. Even though the models are complex, there are tools in giotto-deep to help you understand which feature mostly contributed to the network choice.

In the next cell we use the various features importance techniques on the model.

The output is automatically shipped to the tensorboard, in the image section.

In [ ]:
inter = Interpreter(pipe.model)
inter.feature_importance(next(iter(dl_tr))[0], y=next(iter(dl_tr))[1], n_steps=50)

vs = Visualiser(pipe)
plt = vs.plot_feature_importance(inter)


# Visualise activations and other topological aspects of your model

This concluding section shows you how it is possible to visualise a lot of information (both statistical and topological) on your network in one line thanks to giotto-deep!

In [ ]:
# a batch of data
x, y = next(iter(dl_tr))[0].to(DEVICE), next(iter(dl_tr))[1].to(DEVICE)

# send to tensorboard the model graph!
vs.plot_interactive_model()

# send to tensorboard the 3d plot of a batch!
vs.plot_3d_dataset(n_pts=100)

# send to tensorboard the point cloud, whose coordinates are the activations in each layer, corresponding to the batch x
vs.plot_activations((x, y))

# plot persistence diagrams of the actiivation point clouds
vs.plot_persistence_diagrams((x, y))


In [ ]:
# plot the sampled decision boudnary!
vs.plot_decision_boundary(n_epochs=500, precision=0.7)


In [ ]:
# plot the betti surface of all the activation layers
vs.plot_betti_surface_layers((0, 1), x)
# plot the betti curves of each activation layers
vs.plot_betti_curves_layers((0, 1), x)

In [ ]:
# for a chosen filtration value, compute the evolution of the betti numbers through the layers
fig = vs.plot_betti_numbers_layers(batch=next(iter(dl_tr)), filtration_value=0.3)
